# Ленивые вычисления

Что такое, зачем нужны?

![alt text](http://atkritka.com/upload/iblock/18e/atkritka_1341782872_334.jpg)

## Шаг 1: Осознание проблемы

Допустим, есть два файла заданного формата. Нужно считать эти файлы и записать их содержимое в третий, причем над каждой строкой из файла произвести какую-либо операцию (например, найти все фамилии людей в строке и преобразовать их к общепринятому виду `Фамилия`).

Если попытаться сначала считать файлы, будет не очень хорошо. Не очень хорошо будет потому, что файлы запишутся в оперативную память и ее попросту сожрут. Особенно если файлы большие.

Что делать?

## Шаг 2: Анализ возможных решений

Алгоритм действий

1. открыть файл 1;
2. открыть файл 2;
3. ???
4. PROFIT

![alt text](https://lurkmore.so/images/2/2e/PROFIT.jpg)

## Решение - делать лениво

Концепция **мы ничего не делаем до тех пор, пока нам это действительно сильно не понадобится**.

В нашем случае открываем файлы и читаем по строчке. Обрабатываем по строчке. В ОП не лежит больше одной строчки из файла.

Это и есть концепция ленивых вычислений.

## Пример на Ruby

### Пять первых чисел бесконечной последовательности

In [1]:
(1..Float::INFINITY).lazy.first(5)

[1, 2, 3, 4, 5]

Есть бесконечная последовательность. То есть она не закончится **никогда**. Ruby гарантирует. И при этом по ней можно итерироваться! Но что же произойдет в таком случае? Программа просто зациклится. 

Но что если вдруг нам нужна только какая-то часть этой последовательности? Например, первые 5, 10, 25 или n чисел? Ruby предоставляет возможность сделать это лениво:

1. мы говорим интерпретатору "чувак, тут бесконечная последовательность, но ты ее пока не вычисляй, просто прими к сведению";
2. интерпретатор отвечает "ок, я создаю специальный объект, который уже на низком старте для бесконечного цикла, но пока ничего не делает";

Мы получаем такой объект

In [3]:
(1..Float::INFINITY).lazy

#<Enumerator::Lazy: 1..Infinity>

Мы можем записать его в переменную и хранить там до тех пор, пока он нам не понадобится.

А потом мы можем с помощью этого `Enumerator::Lazy` или взять те самые первые n чисел.

Для чего это собственно нужно? Ведь можно просто сделать цикл.

Это действительно так. На самом деле помимо того, чтобы взять первые n чисел, в Ruby можно лениво идти по последовательности до тех пор, пока выполняется нужное нам условие. То есть можно считать с заданной точностью любые выражения. И делать это довольно удобно.

## Что такое Enumerator?

Это класс, который позволяет итерироваться по коллекции. Вот тут Enumerator используется

In [10]:
[1, 2, 3].each do |x|
  p x
end

1
2
3


[1, 2, 3]

А вот так его можно получить

In [11]:
enum = [1, 2, 3].each
enum.class

Enumerator

In [12]:
p enum.next
enum.next

1


2

In [13]:
enum.next

3

А дальше?

In [14]:
enum.next

StopIteration: iteration reached an end

Enumerator - это отдельный класс, есть еще примесь Enumerable. Enumerator и Enumerable делают по сути одну и ту же работу.

## Как создать свой Enumerator?

Конструктор класса `Enumerator` принимает блок кода, который должен выполняться внутри него. Это может быть как конечный, так и бесконечный цикл.

**Пример конечного цикла** - накопим сумму всех чисел на отрезке [1, 10].

Для этого передадим в Enumerator блок кода, в котором содержится специальная переменная, накапливающая сумму, и цикл, в котором эта сумма обновляется.

Далее мы возвращаем значение из Enumerator. Однако `return` в данном случае писать нельзя, так как Enumerator не может возвращать значения как функция.

In [16]:
ex_enum = Enumerator.new do |yielder|
  sum = 0
  
  (1..10).each do |x|
    sum += x
  end
  
  sum
end

ex_enum

#<Enumerator: #<Enumerator::Generator:0x000000029759f8>:each>

Мы получили объект, который умеет итерироваться по последовательности (пока выполняется внутренний цикл). Более того, этот объект умеет накапливать промежуточные результаты.

Однако чтобы получить конечный результат, нам придется 10 раз вызвать функцию next. Можно скоратить себе работу и вызвать метод each с пустым телом. Он просто будет выполняться, пока выполняется внутренний цикл, поэтому в итоге мы получим ожидаемую сумму - 55.

In [23]:
ex_enum.each {}

1
2
3
4
5
6
7
8
9
10


1..10

**Enumerator на основе бесконечного цикла** должен быть ленивым.

Попробуем накопить сумму квадратов натуральных чисел до произвольного числа с помощью ленивого Enumerator.

In [28]:
factorial = Enumerator.new do |yielder|
  power = 0
  
  (1..Float::INFINITY).each do |number|
    power += number**2
  end
  
  power
end

factorial.lazy.first 2

Interrupt: 

Этот код будет выполняться бесконечно. Причина такого поведения заключается в том, что вызывающей Enumerator программе нужно иметь обратную связь с телом цикла. В противном случае она не будет знать, когда остановить его выполнение.

Грубо говоря это выглядит как возвращение какого-то промежуточного значения с каждой итерации цикла. Но суть в том, что **цикл не прекращает свое выполнение**. Он просто информирует внешнюю программу о своем состоянии.

В Ruby нет способа вернуть значение из цикла, не прервав его выполнение, какой-либо стандартной языковой конструкцией. Для этого используется специальный класс - `Enumerator::Yielder`.

Экземпляр этого класса передается в блок кода при создании Enumerator всегда и неявно, то есть без участия программиста. Возврат значения выглядит следующим образом:

In [30]:
yield_ex = Enumerator.new do |yielder|
  (1..Float::INFINITY).each do |x|
    yielder.yield x
  end
end

yield_ex.lazy.first 10

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

То есть если добавить в пример, который выполнялся бесконечно, yield, все будет работать как ожидается.

In [33]:
factorial = Enumerator.new do |yielder|
  power = 0
  
  (1..Float::INFINITY).each do |number|
    yielder.yield number, power
    power += number**2
  end
end

factorial.lazy.first 5

[[1, 0], [2, 1], [3, 5], [4, 14], [5, 30]]

## Важное замечание - по умолчанию Enumerator не ленивый

Если вы создаете Enumerator, то он выполняться не начнет. Но как только вы напишете `each`, `map`, да и вообще любой метод, который проходится по последовательности до ее конца, Enumerator начнет свое выполнение. И если эта последовательность бесконечная, то он зациклится. Чтобы это исправить, используйте метод lazy. Он говорит интерпретатору о том, что нужно отложить выполнение кода и создать специальный экземпляр класса Enumerator, который называется Enumerator::Lazy и который мы уже видели выше.

## Примеры, где это может пригодиться

### 1

Вычислить сумму ряда $$S = 1 + \sum_{k=1}^\infty 1/{k!}$$ с заданной точностью.

Точное значение - e = 2.718281828...

Так как сумма бесконечная, нам придется использовать бесконечный цикл в Enumerator. 

In [9]:
enum_solve = Enumerator.new do |yielder|
  sum, k_factorial, k = 1, 1, 1
  
  loop do
    yielder.yield sum
    
    k_factorial *= k
    k += 1
    
    sum += 1 / k_factorial.to_f
  end
end

enum_solve.take_while { |sum| (Math::E - sum).abs > 10e-5 }

[1, 2.0, 2.5, 2.6666666666666665, 2.708333333333333, 2.7166666666666663, 2.7180555555555554]

### 2

Сравнить два файла: F и G. Вывести true, если файлы совпадают, и 0 в противном случае.

In [51]:
def equal?(file1, file2)
  result = true
  
  File.open file1, 'r' do |f|
    File.open file2, 'r' do |g|
      f.zip(g).lazy.each do |line_f, line_g|
        unless line_f == line_g
          result = false
          break
        end
      end
    end
  end
  
  result
end

p equal? '2_1', '2_2'
equal? '2_2', '2_2_equal'

false
true


true

### 3

Сформировать программным путем файл F, компоненты которого являются целыми числами. Получить файл G, образованный из файла F исключением повторных вхождений одного и того же числа.

Создаем файл F

In [48]:
File.open 'F', 'w' do |f|
  1488.times do
    f.write (rand() * 1000).to_i.to_s + "\n"
  end
end

1488

Создаем файл G

In [55]:
line_set = Set.new

File.open 'G', 'w' do |g|
  File.open 'F', 'r' do |f|
    f.each_line.lazy.each do |line|
      if line_set.add? line
        g.write line
      end
    end
  end
end

#<File:F (closed)>

### 4

Решить задачу, организовав цикл. Вычислить определенный интеграл $$\int_{\pi/4}^{\pi/3} tg^2(x)dx$$ методом трапеций:
Отрезок интегрирования разбить на n = 20, 30, 40 частей. Точное значение: $$\sqrt{3} - \pi/12 - 1$$

In [1]:
CORRECT = Math.sqrt(3) - Math::PI / 12.0 - 1

0.47025141976972784

In [2]:
def cyclic_solve lower_bound, upper_bound, n_parts
  step = (upper_bound - lower_bound).abs.to_f / n_parts.to_f
  
  prev, cur = lower_bound, lower_bound + step
  
  area = 0
  
  n_parts.times do
    area += step * (Math.tan(cur)**2 + Math.tan(prev)**2) / 2.0Ы
    prev = cur
    cur += step
  end
  
  area
end

:cyclic_solve

In [3]:
cyclic_solve(Math::PI / 4.0, Math::PI / 3.0, 10)

0.47081403105654274

То же самое с помощью Enumerator:

In [ ]:
def enum_solve(n_steps, lower_bound, upper_bound)
  step = (upper_bound - lower_bound).abs.to_f / n_steps.to_f
  prev, cur = lower_bound, lower_bound + step
  area = 0
  
  Enumerator::Lazy.new 1..n_steps do |yielder|
    area += step * (Math.tan(cur)**2 + Math.tan(prev)**2) / 2.0
    prev = cur
    cur += step
    
    yielder.yield area
  end
end

enum_solve(10, Math::PI / 4.0, Math::PI / 3.0).each do |x|
  p x
end

Тут есть замыкание.

Замыкание - это ситуация, когда происходит обращение к объекту внешнего контекста из внутреннего. На иллюстрации это выглядит так:



А в примере это значит вот что. У нас есть функция `enum_solve`, которая возвращает Enumerator. В этой функции объявлены переменные `step`, `prev`, `area`. Эти переменные находятся в контексте функции `enum_solve`, то есть принадлежат этой функции. С другой стороны у нас есть экземпляр класса Enumerator, который пользуется этими переменными. Он их обновляет, использует предыдущие значения, то есть работает с ними так, как будто они находятся в его контексте.

Но фокус в том, что они принадлежат функции `enum_solve` и никакого отношения к Enumerator не имеют. Можно сказать, что это работает потому, что Enumerator тоже находится в контексте этой функции. И это было бы правильно, если бы функция не **возвращала** Enumerator. То есть по логике вещей сразу после того, как конструктор отработает, Enumerator выйдет за пределы нужного нам контекста. Но тем не менее этот код работает. Что же происходит?

Вот тут происходит то самое замыкание. Интерпретатор понимает, что мы хотим работать с переменными, которые нам не принадлежат, и специальным образом создает их копии, которые связаны с нашим конкретным объектом.

## То же самое, но написанное программистом на Perl

In [31]:
def enum_solve(n_steps, lower_bound, upper_bound)
  step = (upper_bound - lower_bound).abs.to_f / n_steps.to_f
  prev, cur = lower_bound, lower_bound + step
  area = 0
  
  Enumerator::Lazy.new 1..n_steps do |yielder|
    area += step * (Math.tan(cur)**2 + Math.tan(prev)**2) / 2.0
    prev = cur
    cur += step
    
    yielder << area
  end.each {|x| area = x}
  
  area
end

enum_solve(10, Math::PI / 4.0, Math::PI / 3.0)

0.47081403105654274

![](sticker.png)